# 基础入门
本快速入门将介绍使用语言模型的基础知识。它将介绍两种不同类型的模型 - LLMs 和 ChatModels。

然后，它将介绍如何使用 PromptTemplates 设置这些模型的输入格式，以及如何使用输出分析器来处理输出。

In [2]:
import os
import openai

from dotenv import find_dotenv, load_dotenv
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

# 如果你需要通过代理端口访问，你需要如下配置
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
os.environ["HTTP_PROXY"] = 'http://127.0.0.1:7890'

## 模型
这两个 llm 对象 chat_model 都是表示特定模型配置的对象。您可以使用 和 others 等 temperature 参数初始化它们，然后传递它们。
它们之间的主要区别在于它们的输入和输出架构。LLM对象将**字符串**作为输入和输出字符串。ChatModel 对象将**消息列表**作为输入并输出消息。

In [3]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI

llm = OpenAI()
chat_model = ChatOpenAI(model="gpt-3.5-turbo")

当我们调用 ChatModel LLM 时，我们可以看到它之间的区别。

In [5]:
from langchain_core.messages import HumanMessage

text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]

response1 = llm.invoke(text)
print(type(response1))
print(response1)
# >> Feetful of Fun

response2 = chat_model.invoke(messages)
print(type(response2))
print(response2)
# >> AIMessage(content="Socks O'Color")

<class 'str'>


"Spectrum Socks" or "Rainbow Footwear Co."
<class 'langchain_core.messages.ai.AIMessage'>
content='Rainbow Sock Co.' response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 22, 'total_tokens': 28}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None}


## Prompt Templates 提示模板

大多数LLM应用程序不会将用户输入直接传递到 LLM.通常，他们会将用户输入添加到更大的文本中，称为提示模板，该文本提供有关特定任务的额外上下文。

与原始字符串格式相比，使用这些格式的优势有几个。您可以“部分”删除变量 - 例如，一次只能格式化部分变量。您可以将它们组合在一起，轻松地将不同的模板组合到一个提示中。

In [6]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product="colorful socks")

'What is a good name for a company that makes colorful socks?'

`PromptTemplate` 还可用于生成消息列表。在这种情况下，提示不仅包含有关内容的信息，还包含每条消息（其角色、在列表中的位置等）。
在这里，最常发生的是 `ChatPromptTemplate` 是 `ChatMessageTemplates` 的列表.
每个 `ChatMessageTemplate` 都包含有关如何格式化的说明 `ChatMessage` - 其角色，然后还有其内容。让我们在下面看一下：

In [7]:
from langchain.prompts.chat import ChatPromptTemplate

template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")

[SystemMessage(content='You are a helpful assistant that translates English to French.'),
 HumanMessage(content='I love programming.')]

ChatPromptTemplates 也可以用其他方式构建 - 有关更多详细信息，请参阅 [prompts](https://python.langchain.com/docs/modules/model_io/prompts)

## Output parsers 输出分析器

`OutputParser` 将语言模型的原始输出转换为可在下游使用的格式。 `OutputParser` 有几种主要类型，包括：

- 将文本转换为 `LLM` 结构化信息 （e.g. JSON）

- 将 `ChatMessage` 转换为字符串

- 将除消息之外的调用返回的额外信息（如 OpenAI 函数调用）转换为字符串。

有关此内容的完整信息，请参阅有关[output_parsers](https://python.langchain.com/docs/modules/model_io/output_parsers)的部分。

在本入门指南中，我们使用一个简单的指南来解析逗号分隔值的列表。|

In [8]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
output_parser.parse("hi, bye")
# >> ['hi', 'bye']

['hi', 'bye']

## Composing with LCEL 使用 LCEL 作曲

我们现在可以将所有这些组合成一个链条。此链将获取输入变量，将这些变量传递给提示模板以创建提示，
将提示传递给语言模型，然后通过（可选）输出解析器传递输出。这是捆绑模块化逻辑的便捷方法。让我们看看它的实际效果吧！

In [ ]:
template = "Generate a list of 5 {text}.\n\n{format_instructions}"

chat_prompt = ChatPromptTemplate.from_template(template)
chat_prompt = chat_prompt.partial(format_instructions=output_parser.get_format_instructions())
output_parser = CommaSeparatedListOutputParser()

chain = chat_prompt | chat_model | output_parser
chain.invoke({"text": "colors"})
# >> ['red', 'blue', 'green', 'yellow', 'orange']